In [1]:
seed = 123

In [2]:
from shutil import copyfile

copyfile(
    src = "/kaggle/input/bdc-p53/p53_helper_functions.py", 
    dst = "/kaggle/working/p53_helpers.py"
)

from p53_helpers import *

# load dataset (no NAs)
X, Y = load_p53_ds()
# sratified 80/20 split
X_train, X_test, Y_train, Y_test = split_p53(X, Y)

Using TensorFlow backend.



Import completed after 2.2 sec

CLASS RATIOS
Training set active classes:   121/24927 (0.485 %)
Test set active classes:       30/6232 (0.481 %)

MATRIX DIMENSIONS
TRAINING SET
. Features:   (24927, 5408)
. Classes:    (24927,)
TEST SET
. Features:   (6232, 5408)
. Classes:    (6232,)


In [3]:
# optimize pipeline w/o resampling
CV = 4

FU = FeatureUnion([
    ("fs_univ", SelectKBest(f_classif, k=20)),
    ("fs_model", SelectFromModel(
        ExtraTreesClassifier(n_estimators=100, random_state=seed), threshold=-np.inf, max_features=125)
    )
])

pipe = Pipeline([
    ("remove_corr", RemoveCorrelatedFeatures(n_remove=250)),
    ("scale", StandardScaler()),
    ("fs", FU),
    #("resample", SVMSMOTE(sampling_strategy=0.01, k_neighbors=10, m_neighbors=10)),
    ("clf", RandomForestClassifier(
        random_state=seed,
        min_samples_leaf=10,
        n_estimators=500,
        max_depth=10
    ))
])


param_grid = dict(
    clf__class_weight=[{0:i, 1:1-i} for i in [0.005, 0.01, 0.05]]
)

GS = GridSearchCV(pipe, param_grid=param_grid, cv=CV, verbose=10, scoring=make_scorer(matthews_corrcoef), n_jobs=-1)
GS.fit(X_train, Y_train)

Fitting 4 folds for each of 3 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:  3.6min remaining:  2.5min
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:  5.5min remaining:  1.8min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  5.6min finished


GridSearchCV(cv=4, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('remove_corr',
                                        RemoveCorrelatedFeatures(n_remove=250)),
                                       ('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('fs',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('fs_univ',
                                                                        SelectKBest(k=20,
                                                                                    score_func=<function f_classif at 0x7f4211c3b488>)),
                                                                       ('fs_model',
                   

In [4]:
results = pd.DataFrame(GS.cv_results_)

results.sort_values('mean_test_score', ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__class_weight,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
1,104.941024,3.263023,0.982495,0.036337,"{0: 0.01, 1: 0.99}","{'clf__class_weight': {0: 0.01, 1: 0.99}}",0.556021,0.470311,0.543913,0.515296,0.521385,0.032988,1
2,120.853757,2.224268,0.735516,0.129009,"{0: 0.05, 1: 0.95}","{'clf__class_weight': {0: 0.05, 1: 0.95}}",0.560178,0.481519,0.521826,0.513982,0.519376,0.027987,2
0,98.509644,2.123114,0.963127,0.067054,"{0: 0.005, 1: 0.995}","{'clf__class_weight': {0: 0.005, 1: 0.995}}",0.569903,0.437151,0.514991,0.489582,0.502907,0.047791,3


In [5]:
results.to_csv(f"/kaggle/working/results_RFC_classification_noRS.csv")

In [6]:
# optimize pipeline w/o resampling
CV = 4

FU = FeatureUnion([
    ("fs_univ", SelectKBest(f_classif, k=20)),
    ("fs_model", SelectFromModel(
        ExtraTreesClassifier(n_estimators=100, random_state=seed), threshold=-np.inf, max_features=125)
    )
])

pipe = Pipeline([
    ("remove_corr", RemoveCorrelatedFeatures(n_remove=250)),
    ("scale", StandardScaler()),
    ("fs", FU),
    ("resample", SVMSMOTE(sampling_strategy=0.01, k_neighbors=10, m_neighbors=10)),
    ("clf", RandomForestClassifier(
        random_state=seed,
        min_samples_leaf=10,
        n_estimators=500,
        max_depth=10
    ))
])


param_grid = dict(
    clf__class_weight=[{0:i, 1:1-i} for i in [0.005, 0.01, 0.025]]
)

GS = GridSearchCV(pipe, param_grid=param_grid, cv=CV, verbose=10, scoring=make_scorer(matthews_corrcoef), n_jobs=-1)
GS.fit(X_train, Y_train)

Fitting 4 folds for each of 3 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:  3.8min remaining:  2.7min
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:  6.0min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  6.0min finished


GridSearchCV(cv=4, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('remove_corr',
                                        RemoveCorrelatedFeatures(n_remove=250)),
                                       ('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('fs',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('fs_univ',
                                                                        SelectKBest(k=20,
                                                                                    score_func=<function f_classif at 0x7f4211c3b488>)),
                                                                       ('fs_model',
                   

In [7]:
results = pd.DataFrame(GS.cv_results_)

results.sort_values('mean_test_score', ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__class_weight,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
1,117.978486,2.011068,1.012072,0.078634,"{0: 0.01, 1: 0.99}","{'clf__class_weight': {0: 0.01, 1: 0.99}}",0.531014,0.481519,0.536972,0.497581,0.511772,0.023032,1
2,129.659666,2.123782,0.751213,0.119624,"{0: 0.025, 1: 0.975}","{'clf__class_weight': {0: 0.025, 1: 0.975}}",0.570976,0.482605,0.495962,0.489371,0.509728,0.035675,2
0,109.108908,2.318116,0.978227,0.053227,"{0: 0.005, 1: 0.995}","{'clf__class_weight': {0: 0.005, 1: 0.995}}",0.596026,0.444684,0.452488,0.466941,0.490035,0.061713,3


In [8]:
results.to_csv(f"/kaggle/working/results_RFC_classification_RS.csv")